In [1]:
%load_ext autoreload
%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns  # Seaborn enhances the aesthetics of matplotlib plots

import pandas as pd
import xarray as xr
import seaborn as sns
import warnings
import logging
# sns.set()

from tempest import casestudy
from tempest import grid
from tempest import storm_tracker
from tempest import joint_distrib
from tempest import handler
from tempest.plots.hist import simple_hist


In [3]:
# settings_path = 'settings/sam_4km_30min_30d.yaml'
# "settings/arpege_summer_30d.yaml",
#"settings/fv3_summer_30d.yaml"  ,
settings_paths = [ "settings/arpege_summer_30d.yaml", "settings/fv3_summer_30d.yaml", "settings/ifs_summer_30d.yaml", 
                    "settings/mpas_summer_30d.yaml", "settings/nicam_summer_30d.yaml", "settings/obs_summer_30d.yaml", 
                    "settings/sam_summer_30d.yaml", "settings/um_summer_30d.yaml"]
# settings_paths = ["settings/nicam_summer_30d.yaml",  "settings/um_summer_30d.yaml"]
# settings_paths = [ "settings/sam_4km_30min_30d.yaml"]

hdlrs = [handler.Handler(settings_path) for settings_path in settings_paths]
css = [casestudy.CaseStudy(hdlr, overwrite = False ,verbose = False) for hdlr in hdlrs]
grs = [grid.Grid(cs, fast = True, overwrite= False, verbose_steps = False, verbose = False) for cs in css]
jds = [joint_distrib.JointDistribution(gr, None, var_id_1 = "mean_unweighted_Prec", var_id_2 = "cond_alpha_85_Prec", 
        nbpd = 20,  nd=5, overwrite = True, dist_mask = False) for gr in grs]

lm = grs[0].get_landmask()
ocean = lm.Landmask == 0
ocean = ocean.values[:,:,np.newaxis]

Data loaded from /homedata/mcarenso/tempest/ARPEGE_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/FV3_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/IFS_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/MPAS_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/NICAM_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/OBS_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/SAM_lowRes_Tropics/var_id_days_i_t.json
Data loaded from /homedata/mcarenso/tempest/UM_lowRes_Tropics/var_id_days_i_t.json
Found grid attributes file , so loading /homedata/mcarenso/tempest/ARPEGE_lowRes_Tropics/grid_attributes.pkl instead of computing
Found grid attributes file , so loading /homedata/mcarenso/tempest/FV3_lowRes_Tropics/grid_attributes.pkl instead of computing
Found grid attributes file , so loading /homedata/mcarenso/tempest

/net/nfs/tools/u20/Python/miniconda3_py310_22.11.1-1/envs/pangeo-meso-2023.04.15/lib/python3.10/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


Distribs have been recomputed because overwrite is set to True
Overwrite set to true, so computing basics and saving them


/net/nfs/tools/u20/Python/miniconda3_py310_22.11.1-1/envs/pangeo-meso-2023.04.15/lib/python3.10/site-packages/numpy/lib/histograms.py:1123: RuntimeWarning: invalid value encountered in divide
  hist = hist / dedges[i].reshape(shape)


Distribs have been recomputed because overwrite is set to True
Overwrite set to true, so computing basics and saving them
Distribs have been recomputed because overwrite is set to True
Overwrite set to true, so computing basics and saving them


In [ ]:
import os 
cloud_cond_var_id = "sliding_MCS_cond_Prec_15"
sigma = jd.prec.Sigma_cond_alpha_85_Prec
mcs_cond = gr.get_var_id_ds(cloud_cond_var_id)
sigma_mcs = mcs_cond.Rel_surface.sum(axis = 3) ## add this
mcs_cond["intersection_"+cloud_cond_var_id] = sigma_mcs/sigma ## add this 
mcs_cond_file = gr.get_var_ds_file(cloud_cond_var_id)
os.remove(mcs_cond_file)
mcs_cond.to_netcdf(mcs_cond_file)

In [4]:
# for gr in grs:
#     st = storm_tracker.StormTracker(gr, label_var_id = "MCS_label", overwrite_storms = True, overwrite = False)
#     # st = storm_tracker.StormTracker(gr, label_var_id = "MCS_Feng", overwrite_storms = True, overwrite = False)


making ds storms ...

Label wise attributes  ['DCS_number', 'INT_TbMin', 'INT_UTC_timeEnd', 'INT_UTC_timeInit', 'INT_classif', 'INT_classif_JIRAK', 'INT_classif_MADDOX', 'INT_distance', 'INT_duration', 'INT_latEnd', 'INT_latInit', 'INT_latmax', 'INT_latmin', 'INT_localtime_End', 'INT_localtime_Init', 'INT_lonEnd', 'INT_lonInit', 'INT_lonmax', 'INT_lonmin', 'INT_qltyDCS', 'INT_surfcumkm2_241K', 'INT_surfmaxPix_241K', 'INT_surfmaxkm2_200K', 'INT_surfmaxkm2_210K', 'INT_surfmaxkm2_220K', 'INT_surfmaxkm2_241K', 'INT_velocityAvg']
Label's Lifecycle wise attributes  ['LC_UTC_time', 'LC_ecc_220K', 'LC_ecc_241K', 'LC_lat', 'LC_localtime', 'LC_lon', 'LC_orientation_220K', 'LC_orientation_241K', 'LC_smajor_220K', 'LC_smajor_241K', 'LC_sminor_220K', 'LC_sminor_241K', 'LC_surfPix_210K', 'LC_surfPix_241K', 'LC_surfkm2_200K', 'LC_surfkm2_210K', 'LC_surfkm2_220K', 'LC_surfkm2_241K', 'LC_tb_90th', 'LC_tbavg_200K', 'LC_tbavg_208K', 'LC_tbavg_241K', 'LC_tbmin', 'LC_velocity', 'LC_x', 'LC_y', 'QCgeo_IRima